# Phân tích One-way ANOVA: mức độ hài lòng TMĐT giữa các thế hệ

Notebook này minh họa quy trình chuẩn của một bài nghiên cứu định lượng:

1. Đọc dữ liệu khảo sát
2. Tạo biến thế hệ (`generation`) và điểm hài lòng tổng hợp (`satisfaction`)
3. Thống kê mô tả và vẽ biểu đồ
4. Kiểm tra giả định trước khi dùng one-way ANOVA (chuẩn, đồng nhất phương sai)
5. Thực hiện ANOVA
6. Kiểm định hậu nghiệm Tukey HSD
7. Gợi ý cách viết kết luận cho bài nghiên cứu


In [ ]:
# 1. Import thư viện cần thiết
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import shapiro, levene, f_oneway
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

sns.set(style="whitegrid", font_scale=1.1)
plt.rcParams["figure.figsize"] = (8, 5)


## Bước 1 – Đọc dữ liệu khảo sát

- File dữ liệu mẫu: `survey_data.csv`
- Các cột chính:
  - `year_of_birth`: năm sinh người trả lời
  - `Q1`–`Q10`: các câu hỏi thang Likert 1–5 về hài lòng TMĐT


In [ ]:
# Đọc dữ liệu từ file CSV
# Đảm bảo file survey_data.csv nằm cùng thư mục với notebook này

df = pd.read_csv("survey_data.csv")
print("5 dòng đầu dữ liệu:")
display(df.head())


## Bước 2 – Tạo biến thế hệ (`generation`) và điểm hài lòng (`satisfaction`)

- Mã hóa năm sinh thành 3 thế hệ: Gen X, Millennials, Gen Z.
- Tạo biến `satisfaction` = trung bình điểm Q1–Q10 cho mỗi người trả lời.


In [ ]:
# Hàm mã hóa năm sinh thành thế hệ

def map_generation(year):
    if 1965 <= year <= 1980:
        return "Gen X"
    elif 1981 <= year <= 1996:
        return "Millennials"
    elif 1997 <= year <= 2012:
        return "Gen Z"
    else:
        return "Khác"

# Nếu chưa có cột generation thì tạo mới
if "generation" not in df.columns:
    df["generation"] = df["year_of_birth"].apply(map_generation)

# Giữ lại 3 nhóm chính
df = df[df["generation"].isin(["Gen X", "Millennials", "Gen Z"])].copy()

# Tạo biến satisfaction từ Q1..Q10
item_cols = [f"Q{i}" for i in range(1, 11)]

df["satisfaction"] = df[item_cols].mean(axis=1)

print("5 dòng đầu sau khi tạo generation và satisfaction:")
display(df[["year_of_birth", "generation", "satisfaction"]].head())

print("Phân bố số lượng theo thế hệ:")
display(df["generation"].value_counts())


## Bước 3.1 – Đánh giá độ tin cậy thang đo (Cronbach's Alpha)

Trước khi phân tích ANOVA, cần kiểm tra độ tin cậy của thang đo (10 mục hỏi Q1-Q10).

In [ ]:
# Hàm tính Cronbach's Alpha
def cronbach_alpha(df_items):
    """Tính Cronbach's Alpha cho thang đo"""
    item_variances = df_items.var(axis=0, ddof=1)
    total_variance = df_items.sum(axis=1).var(ddof=1)
    n_items = df_items.shape[1]
    return n_items / (n_items - 1) * (1 - item_variances.sum() / total_variance)

# Tính Cronbach's Alpha
alpha_overall = cronbach_alpha(df[item_cols])
print(f"Cronbach's Alpha (toàn bộ mẫu) = {alpha_overall:.4f}")

# Tính alpha cho từng nhóm
print("\nCronbach's Alpha theo từng thế hệ:")
for gen in ["Gen X", "Millennials", "Gen Z"]:
    df_gen = df[df["generation"] == gen]
    alpha_gen = cronbach_alpha(df_gen[item_cols])
    print(f"  {gen:12s}: α = {alpha_gen:.4f}")

# Đánh giá
if alpha_overall >= 0.9:
    print(f"\n✅ Độ tin cậy XUẤT SẮC (α ≥ 0.9)")
elif alpha_overall >= 0.8:
    print(f"\n✅ Độ tin cậy TỐT (0.8 ≤ α < 0.9)")
elif alpha_overall >= 0.7:
    print(f"\n⚠️  Độ tin cậy CHẤP NHẬN ĐƯỢC (0.7 ≤ α < 0.8)")
else:
    print(f"\n❌ Độ tin cậy THẤP (α < 0.7) - Thang đo cần xem xét lại!")

## Bước 3.2 – Thống kê mô tả và biểu đồ

## Bước 4 – Thực hiện One-way ANOVA

- Kiểm định xem trung bình `satisfaction` có khác nhau giữa 3 thế hệ.
- H0: μ(Gen Z) = μ(Millennials) = μ(Gen X).
- H1: Có ít nhất một cặp trung bình khác nhau.

In [ ]:
# ANOVA bằng scipy và bảng ANOVA chi tiết bằng statsmodels

scores_z = df.loc[df["generation"] == "Gen Z", "satisfaction"]
scores_m = df.loc[df["generation"] == "Millennials", "satisfaction"]
scores_x = df.loc[df["generation"] == "Gen X", "satisfaction"]

f_stat, p_anova = f_oneway(scores_z, scores_m, scores_x)
print(f"ANOVA (scipy) -> F = {f_stat:.3f}, p-value = {p_anova:.6f}")

model = ols('satisfaction ~ C(generation)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print("\nBảng ANOVA (statsmodels):")
display(anova_table)

## Bước 5 – Kiểm tra giả định ANOVA

Sau khi fit mô hình ANOVA, cần kiểm tra các giả định:

### 5.1. Giả định phân phối chuẩn của residuals

- Giả định: Residuals (phần dư) có phân phối chuẩn N(0, σ²)
- Residual = y_i - ȳ_nhóm_i (sai lệch giữa giá trị quan sát và trung bình nhóm)
- Kiểm định: Shapiro-Wilk và QQ plot
- H0: Residuals có phân phối chuẩn
- Nếu p > 0.05 → Không bác bỏ H0 (chấp nhận giả định)

In [ ]:
# Lấy residuals từ mô hình ANOVA
residuals = model.resid

# Kiểm định Shapiro-Wilk cho residuals
stat_shapiro, p_shapiro = shapiro(residuals)
print(f"Shapiro-Wilk test (residuals):")
print(f"  W = {stat_shapiro:.4f}")
print(f"  p-value = {p_shapiro:.4e}")

if p_shapiro > 0.05:
    print("  => ✅ KHÔNG bác bỏ H0: Residuals có phân phối chuẩn")
else:
    print(f"  => ⚠️  Bác bỏ H0: Residuals vi phạm giả định chuẩn (p={p_shapiro:.4e})")
    print(f"  Tuy nhiên, với n={len(df)} (lớn) và cân bằng, ANOVA vẫn robust.")
    
# QQ plot để kiểm tra trực quan
import scipy.stats as stats

fig, ax = plt.subplots(figsize=(8, 6))
stats.probplot(residuals, dist="norm", plot=ax)
ax.set_title("Q-Q Plot: Kiểm tra phân phối chuẩn của Residuals", fontsize=14, fontweight='bold')
ax.set_xlabel("Theoretical Quantiles (Phân vị lý thuyết)", fontsize=12)
ax.set_ylabel("Sample Quantiles (Phân vị mẫu)", fontsize=12)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig("fig4_qqplot_residuals.png", dpi=300, bbox_inches='tight')
plt.show()

print("\n💡 Cách đọc QQ plot:")
print("   - Điểm nằm SÁT đường thẳng → Residuals phân phối chuẩn")
print("   - Điểm LỆCH ở 2 đầu → Vi phạm nhẹ, chấp nhận được với n lớn")

### 5.2. Giả định đồng nhất phương sai (Homogeneity of Variance)

- Giả định: Phương sai của dữ liệu trong các nhóm bằng nhau
- Var(Gen X) = Var(Millennials) = Var(Gen Z)
- Kiểm định: Levene test
- H0: Phương sai các nhóm bằng nhau
- Nếu p > 0.05 → Không bác bỏ H0 (chấp nhận giả định)

In [ ]:
# Kiểm tra phương sai từng nhóm
print("Phương sai (variance) từng nhóm:")
for gen in ["Gen X", "Millennials", "Gen Z"]:
    scores = df[df["generation"] == gen]["satisfaction"]
    var = scores.var(ddof=1)
    std = scores.std(ddof=1)
    print(f"  {gen:12s}: Var = {var:.4f}, SD = {std:.4f}")

# Tỷ lệ phương sai max/min
variances = [df[df["generation"] == gen]["satisfaction"].var(ddof=1) 
             for gen in ["Gen X", "Millennials", "Gen Z"]]
ratio = max(variances) / min(variances)
print(f"\nTỷ lệ phương sai max/min = {ratio:.2f}")
print(f"  (Nếu < 3 → Chấp nhận được theo quy tắc thực nghiệm)")

# Levene test
stat_levene, p_levene = levene(scores_x, scores_m, scores_z)
print(f"\nLevene test:")
print(f"  F = {stat_levene:.2f}")
print(f"  p-value = {p_levene:.4e}")

if p_levene > 0.05:
    print("  => ✅ KHÔNG bác bỏ H0: Phương sai các nhóm bằng nhau")
else:
    print(f"  => ⚠️  Bác bỏ H0: Phương sai các nhóm KHÁC NHAU (p={p_levene:.4e})")
    print(f"  Tuy nhiên, tỷ lệ={ratio:.2f}<3 và n cân bằng → ANOVA vẫn robust")
    print(f"  Nên kiểm chứng bằng Welch ANOVA hoặc Kruskal-Wallis")

## Bước 6 – Kiểm định hậu nghiệm Tukey HSD

- Thực hiện khi ANOVA có ý nghĩa (p-value < 0.05).
- Mục tiêu: xem cặp thế hệ nào khác nhau về mức hài lòng.

In [ ]:
if p_anova < 0.05:
    print("Vì p-value ANOVA < 0.05 nên thực hiện kiểm định hậu nghiệm Tukey HSD:")
    tukey = pairwise_tukeyhsd(endog=df['satisfaction'],
                              groups=df['generation'],
                              alpha=0.05)
    print(tukey)
else:
    print("p-value ANOVA >= 0.05: không đủ bằng chứng khác biệt giữa các thế hệ, thường không cần Tukey.")


## Bước 7 – Gợi ý cách viết kết luận

Cell dưới đây in ra gợi ý kết luận để tham khảo khi viết báo cáo (nêu F, p và diễn giải H0).

In [ ]:
alpha = 0.05

# Lấy bậc tự do từ bảng ANOVA
df_between = int(anova_table.loc['C(generation)', 'df'])
df_within = int(anova_table.loc['Residual', 'df'])

print("--- GỢI Ý KẾT LUẬN ---")
print(f"Kết quả ANOVA một nhân tố cho thấy F({df_between}, {df_within}) = {f_stat:.2f}, p = {p_anova:.4f}.")

if p_anova < alpha:
    print("Vì p < 0.05 nên bác bỏ giả thuyết H0: có sự khác biệt có ý nghĩa thống kê về mức độ hài lòng TMĐT giữa ít nhất hai thế hệ.")
    print("Học sinh có thể dựa vào bảng Tukey để mô tả cụ thể cặp thế hệ nào khác nhau (ví dụ: Gen Z > Gen X).")
else:
    print("Vì p ≥ 0.05 nên không bác bỏ giả thuyết H0: không tìm thấy bằng chứng đủ mạnh về sự khác biệt mức độ hài lòng TMĐT giữa các thế hệ.")


## Bước 8 – Kiểm định Kruskal-Wallis (kiểm chứng, phi tham số)

Để kiểm tra độ bền vững của kết luận khi giả định bị vi phạm nhẹ, ta dùng thêm kiểm định phi tham số **Kruskal-Wallis** (so sánh ba nhóm dựa trên thứ hạng, không yêu cầu phân phối chuẩn).

In [ ]:
from scipy.stats import kruskal

# Kruskal–Wallis cho ba nhóm thế hệ
H_stat, p_kw = kruskal(scores_x, scores_m, scores_z)
print(f"Kruskal–Wallis: H(2) = {H_stat:.3f}, p = {p_kw:.3g}")

if p_kw < 0.05:
    print("=> Bác bỏ H0: có sự khác biệt có ý nghĩa thống kê về mức độ hài lòng giữa ít nhất hai thế hệ (theo kiểm định phi tham số).")
else:
    print("=> Không bác bỏ H0: không tìm thấy khác biệt có ý nghĩa (theo kiểm định phi tham số).")
